# E-commerce PostgreSQL Project
#### In this project, I'll be exploring how to design, implement, and maintain efficient and scalable database applications.

### In this project, I'll be tackling: 
- Writing efficient SQL queries: Writing optimized queries to retrieve data from databases in the most efficient way possible. I'll be doingindexing, query optimization, and performance tuning.

- Structuring large datasets using partitioning: I'll be exploring partitioning techniques like range, list, and hash partitioning to break up large datasets into smaller, more manageable pieces. This will help me manage big datasets effectively and improve the performance and scalability of my database applications.

- Handling database errors: I'll be learning how to handle a wide variety of database errors, including techniques like logging, retrying, and rollback strategies. This will ensure that my database applications are reliable and robust, even in the face of errors and failures.

- Using design patterns for resiliency: I'll be incorporating design patterns that improve the resiliency of my database applications, such as circuit breaker, retry, and fallback patterns. These patterns will help me handle failures and keep my applications up and running smoothly, even in challenging situations.

### Topics covered will include:

- Writing efficient SQL queries
- Dataset partitioning
- Handling database errors
- Using design patterns for resiliency

Join me to learn best practices for developing highly available and scalable database applications. Let's dive in!

In [43]:
#We'll start by importing necessary modules 
#Backend was already made with setting up tables within pgadmin 4 & Postgresql
import psycopg2
import os

In [45]:
USER = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']

In [46]:
USER

'postgres'

In [47]:
len(PASSWORD)

6

In [48]:
#We'll connect to the database
conn = psycopg2.connect(database='ecomm',
                        user=USER,
                        password=PASSWORD,
                        port=5432)

In [49]:
conn

<connection object at 0x000001D3AC283560; dsn: 'user=postgres password=xxx dbname=ecomm port=5432', closed: 0>

In [50]:
#We'll create a cursor to be able to connect to the column/table we want to see
#and we'll filter from there
def getProductsByType(db_conn, prod_type):
    cursor = db_conn.cursor()
    sql_string = "SELECT * FROM products WHERE product_type = %s"
    cursor.execute(sql_string, [prod_type])
    return(cursor_fetchall())


In [51]:
def getProductsByType(db_conn, prod_type):
    cursor = db_conn.cursor()
    sql_string = "SELECT * FROM products WHERE product_type = %s"
    cursor.execute(sql_string, [prod_type])
    return cursor.fetchall()


In [52]:
#We'll create a variable to see all the products under the "Oven" category
all_ovens = getProductsByType(conn, 'oven')

In [53]:
#print variable
all_ovens

[(1, 'Reflector oven', 'oven'),
 (2, 'Convection microwave', 'oven'),
 (15, 'Masonry oven', 'oven'),
 (24, 'Microwave oven', 'oven'),
 (32, 'Russian oven', 'oven'),
 (33, 'Clome oven', 'oven'),
 (34, 'Convection oven', 'oven'),
 (37, 'Beehive oven', 'oven'),
 (38, 'Toaster and toaster ovens', 'oven'),
 (41, 'Self-cleaning oven', 'oven'),
 (42, 'Wood-fired oven', 'oven'),
 (43, 'Kitchener range', 'oven'),
 (47, 'Oven', 'oven'),
 (48, 'Hot box (appliance)', 'oven'),
 (58, 'Tabun oven', 'oven'),
 (67, 'Chorkor oven', 'oven'),
 (72, 'Communal oven', 'oven'),
 (74, 'Halogen oven', 'oven'),
 (78, 'Stove', 'oven'),
 (80, 'Earth oven', 'oven')]

In [54]:
len(all_ovens)

20

In [55]:
#This is what's called an "N+1" query
#The N+1 query is a very innefficient way of writing this code to get tables from the db
#Therefore, I'll be creating a more efficient way of pulling the data. As you can see below, 
#the data is in chunks, and isn't as readable or useful as it can be.
cursor = conn.cursor()
fetchall_cnt = 0

for oven in all_ovens:
    oven_id = oven[0]
    sql_string = "SELECT * FROM product_orders WHERE product_id = %s"
    cursor.execute(sql_string, [oven_id])
    oven_order = cursor.fetchall()
    fetchall_cnt += 1
    print(oven_order, "\n")
    print("fetchall_cnt:", fetchall_cnt)

[(4, 1, 1, Decimal('296.47')), (23, 1, 4, Decimal('30.99')), (57, 1, 4, Decimal('124.53')), (63, 1, 2, Decimal('131.79'))] 

fetchall_cnt: 1
[(5, 2, 2, Decimal('270.24')), (21, 2, 2, Decimal('196.39'))] 

fetchall_cnt: 2
[(9, 15, 3, Decimal('53.26')), (38, 15, 2, Decimal('299.62')), (41, 15, 2, Decimal('158.61'))] 

fetchall_cnt: 3
[(3, 24, 5, Decimal('131.20')), (32, 24, 5, Decimal('60.54')), (36, 24, 5, Decimal('93.13')), (49, 24, 4, Decimal('228.74')), (81, 24, 2, Decimal('283.90'))] 

fetchall_cnt: 4
[(24, 32, 3, Decimal('70.57'))] 

fetchall_cnt: 5
[(38, 33, 4, Decimal('236.11')), (40, 33, 1, Decimal('227.86')), (61, 33, 2, Decimal('45.44')), (82, 33, 3, Decimal('29.96'))] 

fetchall_cnt: 6
[(12, 34, 3, Decimal('251.16')), (28, 34, 4, Decimal('72.18')), (69, 34, 5, Decimal('73.06')), (71, 34, 2, Decimal('75.19'))] 

fetchall_cnt: 7
[(35, 37, 4, Decimal('211.71')), (49, 37, 1, Decimal('162.63')), (51, 37, 5, Decimal('271.15')), (61, 37, 2, Decimal('256.34'))] 

fetchall_cnt: 8
[(56

In [56]:
#Optimized code to fetch all data in one block
def getProductOrderByType(db_conn, prod_type):
    cursor = db_conn.cursor()
    sql_string = "SELECT po.*  \
                    FROM product_orders po \
                    INNER JOIN products p      \
                    ON po.product_id = p.product_id   \
                    WHERE p.product_type = %s"
    cursor.execute(sql_string, [prod_type])
    return(cursor.fetchall())
    

In [57]:
#calling new variable that was just created
getProductOrderByType(conn, 'oven')

[(2, 58, 5, Decimal('102.97')),
 (2, 80, 1, Decimal('181.54')),
 (3, 24, 5, Decimal('131.20')),
 (4, 1, 1, Decimal('296.47')),
 (5, 2, 2, Decimal('270.24')),
 (8, 42, 2, Decimal('99.52')),
 (9, 15, 3, Decimal('53.26')),
 (12, 34, 3, Decimal('251.16')),
 (12, 48, 4, Decimal('20.08')),
 (14, 72, 2, Decimal('118.64')),
 (16, 47, 1, Decimal('246.68')),
 (18, 42, 1, Decimal('281.35')),
 (21, 2, 2, Decimal('196.39')),
 (21, 48, 5, Decimal('52.86')),
 (23, 1, 4, Decimal('30.99')),
 (23, 78, 3, Decimal('245.55')),
 (24, 32, 3, Decimal('70.57')),
 (24, 78, 4, Decimal('21.83')),
 (25, 58, 4, Decimal('85.03')),
 (26, 80, 3, Decimal('30.75')),
 (27, 74, 1, Decimal('202.45')),
 (28, 34, 4, Decimal('72.18')),
 (28, 58, 2, Decimal('289.46')),
 (32, 24, 5, Decimal('60.54')),
 (35, 37, 4, Decimal('211.71')),
 (36, 24, 5, Decimal('93.13')),
 (38, 15, 2, Decimal('299.62')),
 (38, 33, 4, Decimal('236.11')),
 (38, 58, 3, Decimal('255.12')),
 (40, 33, 1, Decimal('227.86')),
 (40, 58, 2, Decimal('247.79')),


## Testing Database with a Certain Product

#### Now I'll write a function that executes a SQL query to retreive all product orders. 
- I want to specify product type, for this example, we'll search for "specialty". 
- I also want the product to come back with a quantity larger than 4

In [79]:
#Below is efficient code, not N+1 styles that would execute inefficient tables
#We'll name getProductOrderByType with a "2" to differentiate more easily with the less efficient code above
def getProductOrderByType2(db_conn, prod_type):
    cursor = db_conn.cursor()
    sql_string = "SELECT po.*  \
                    FROM product_orders po \
                    INNER JOIN products p      \
                    ON po.product_id = p.product_id   \
                    WHERE p.product_type = %s    \
                    AND po.quantity > 4"
    cursor.execute(sql_string, [prod_type])
    return(cursor.fetchall())

In [67]:
getProductOrderByType2(conn, 'specialty')

[(18, 45, 5, Decimal('42.58')),
 (41, 39, 5, Decimal('256.48')),
 (42, 7, 5, Decimal('201.44')),
 (56, 18, 5, Decimal('252.06')),
 (64, 7, 5, Decimal('261.31'))]

# Parameterize Quantity
#### Now I want to make the above code block more useful by making the quantity parameter be any number I may choose in the future, with the %s function 

In [69]:
def getProductOrderByType3(db_conn, prod_type):
    cursor = db_conn.cursor()
    sql_string = "SELECT po.*  \
                    FROM product_orders po \
                    INNER JOIN products p      \
                    ON po.product_id = p.product_id   \
                    WHERE p.product_type = %s    \
                    AND po.quantity > %s"
    cursor.execute(sql_string, [prod_type,3])
    return(cursor.fetchall())

In [70]:
#I'll get the same results, but it's parameterized now
getProductOrderByType3(conn, 'specialty')

[(18, 45, 5, Decimal('42.58')),
 (23, 39, 4, Decimal('107.69')),
 (41, 39, 5, Decimal('256.48')),
 (42, 7, 5, Decimal('201.44')),
 (55, 45, 4, Decimal('97.79')),
 (56, 18, 5, Decimal('252.06')),
 (64, 7, 5, Decimal('261.31')),
 (79, 59, 4, Decimal('155.79'))]

In [88]:
#I'll try a different quantity in the parameter
def getProductOrderByType3(db_conn, prod_type):
    cursor = db_conn.cursor()
    sql_string = "SELECT po.*  \
                    FROM product_orders po \
                    INNER JOIN products p      \
                    ON po.product_id = p.product_id   \
                    WHERE p.product_type = %s    \
                    AND po.quantity > %s"
    cursor.execute(sql_string, [prod_type,2])
    return(cursor.fetchall())

In [89]:
#And we see differnt results according to the changed quantity
getProductOrderByType3(conn, 'specialty')

[(1, 59, 3, Decimal('22.07')),
 (9, 39, 3, Decimal('140.78')),
 (18, 45, 5, Decimal('42.58')),
 (23, 39, 4, Decimal('107.69')),
 (41, 39, 5, Decimal('256.48')),
 (42, 7, 5, Decimal('201.44')),
 (47, 21, 3, Decimal('73.83')),
 (55, 45, 4, Decimal('97.79')),
 (56, 18, 5, Decimal('252.06')),
 (62, 45, 3, Decimal('269.38')),
 (64, 7, 5, Decimal('261.31')),
 (78, 27, 3, Decimal('119.16')),
 (79, 59, 4, Decimal('155.79'))]

In [90]:
#Trying a similar code with different quantity and *product* to test
def getProductOrderByType3(db_conn, prod_type):
    cursor = db_conn.cursor()
    sql_string = "SELECT po.*  \
                    FROM product_orders po \
                    INNER JOIN products p      \
                    ON po.product_id = p.product_id   \
                    WHERE p.product_type = %s    \
                    AND po.quantity > %s"
    cursor.execute(sql_string, [prod_type,2])
    return(cursor.fetchall())

In [91]:
getProductOrderByType3(conn, 'cooker')

[(1, 25, 4, Decimal('63.59')),
 (14, 73, 3, Decimal('29.62')),
 (25, 82, 5, Decimal('203.47')),
 (32, 23, 5, Decimal('114.08')),
 (35, 82, 4, Decimal('208.97')),
 (38, 51, 4, Decimal('163.11')),
 (40, 9, 4, Decimal('84.47')),
 (40, 23, 5, Decimal('258.55')),
 (43, 9, 5, Decimal('230.13')),
 (44, 82, 3, Decimal('208.36')),
 (49, 75, 3, Decimal('102.44')),
 (54, 75, 4, Decimal('165.38')),
 (56, 19, 5, Decimal('39.05')),
 (59, 75, 3, Decimal('247.46')),
 (74, 4, 5, Decimal('147.32')),
 (80, 9, 5, Decimal('111.15'))]